In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población media anual,513170
1,Huelva,Confirmados PDIA,57728
2,Huelva,Confirmados PDIA 14 días,6696
3,Huelva,Tasa PDIA 14 días,"1304,8307578385331"
4,Huelva,Confirmados PDIA 7 días,3209
5,Huelva,Tasa PDIA 7 dias,"625,3288383966327"
6,Huelva,Total Confirmados,57941
7,Huelva,Curados,47754
8,Huelva,Fallecidos,427


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  57728.0


/tmp/ipykernel_31976/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  16531.0


/tmp/ipykernel_31976/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]

/tmp/ipykernel_31976/242626003.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_31976/242626003.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_31976/242626003.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 140 personas en los últimos 7 días 

Un positivo PCR cada 66 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población media anual' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Rename columns
tabla.columns = ['Confirmados PDIA', 'Confirmados PDIA 14 días', 'Confirmados PDIA 7 días', 'Población']

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,57728.0,6696.0,3209.0,513170.0,625.328838,1304.830758,612.0
Huelva-Costa,33985.0,3899.0,1803.0,289548.0,622.694683,1346.581569,383.0
Huelva (capital),16531.0,2189.0,1031.0,143837.0,716.783581,1521.861552,245.0
Condado-Campiña,17357.0,1605.0,859.0,156231.0,549.826859,1027.324923,114.0
Sierra de Huelva-Andévalo Central,5939.0,1178.0,538.0,67391.0,798.326186,1748.007894,113.0
Valverde del Camino,1329.0,360.0,139.0,12750.0,1090.196078,2823.529412,38.0
Aljaraque,2217.0,339.0,137.0,21474.0,637.980814,1578.653255,37.0
Lepe,3413.0,238.0,126.0,27880.0,451.936872,853.658537,33.0
Cartaya,2561.0,187.0,114.0,20083.0,567.644276,931.135786,23.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
San Silvestre de Guzmán,75.0,46.0,8.0,614.0,1302.931596,7491.856678,0.0
Campofrío,84.0,22.0,16.0,713.0,2244.039271,3085.553997,4.0
Galaroza,106.0,42.0,18.0,1382.0,1302.460203,3039.073806,1.0
Valverde del Camino,1329.0,360.0,139.0,12750.0,1090.196078,2823.529412,38.0
Puerto Moral,26.0,7.0,3.0,278.0,1079.136691,2517.985612,1.0
Fuenteheridos,45.0,16.0,3.0,709.0,423.131171,2256.699577,0.0
Jabugo,138.0,48.0,38.0,2260.0,1681.415929,2123.893805,7.0
Cerro de Andévalo (El),253.0,48.0,27.0,2327.0,1160.292222,2062.741728,8.0
Escacena del Campo,302.0,47.0,18.0,2287.0,787.057280,2055.094010,2.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Paymogo,98.0,8.0,1.0,1162.0,86.058520,688.468158,0.0,0.125000
Sanlúcar de Guadiana,42.0,6.0,1.0,403.0,248.138958,1488.833747,0.0,0.166667
San Silvestre de Guzmán,75.0,46.0,8.0,614.0,1302.931596,7491.856678,0.0,0.173913
Fuenteheridos,45.0,16.0,3.0,709.0,423.131171,2256.699577,0.0,0.187500
Puebla de Guzmán,228.0,41.0,9.0,3092.0,291.073739,1326.002587,2.0,0.219512
Higuera de la Sierra,92.0,15.0,4.0,1291.0,309.837335,1161.890008,0.0,0.266667
Aracena,1045.0,159.0,58.0,8255.0,702.604482,1926.105391,10.0,0.364780
Escacena del Campo,302.0,47.0,18.0,2287.0,787.057280,2055.094010,2.0,0.382979
Valverde del Camino,1329.0,360.0,139.0,12750.0,1090.196078,2823.529412,38.0,0.386111
